# CSE6242 - HW3 - Q1

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove any comment that says "#export" because that will crash the autograder in Gradescope. We use this comment to export your code in these cells for grading.
</div>

Pyspark Imports

In [1]:
#export
### DO NOT MODIFY THIS CELL ###
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.functions import hour, when, col, date_format, to_timestamp, ceil, coalesce

Initialize PySpark Context

In [2]:
### DO NOT MODIFY THIS CELL ###
sc = pyspark.SparkContext(appName="HW3-Q1")
sqlContext = SQLContext(sc)

25/03/11 03:06:41 WARN Utils: Your hostname, DESKTOP-P0T2SUH resolves to a loopback address: 127.0.1.1; using 172.31.16.68 instead (on interface eth0)
25/03/11 03:06:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/11 03:06:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/usr/local/lib/python3.9/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Define function for loading data

In [3]:
### DO NOT MODIFY THIS CELL ###
def load_data():
    df = sqlContext.read.option("header",True) \
     .csv("yellow_tripdata_2019-01_short.csv")
    return df

### Q1.1

Perform data casting to clean incoming dataset

In [4]:
#export
#print(pyspark.__version__)
#df.printSchema()
#df.show(5)

#df = load_data()
#print("Before Casting columns")
#df.printSchema()


def clean_data(df):
    '''
    input: df a dataframe
    output: df a dataframe with the all the original columns
    '''
    
    # START YOUR CODE HERE ---------
    #columns_to_cast = ['passenger_count', 'total_amount', 'tip_amount', 'trip_distance', 'fare_amount', 'tpep_pickup_datetime', 'tpep_dropoff_datetime']

    #for columns in columns_to_cast:
    from pyspark.sql.types import IntegerType, FloatType, TimestampType
    from pyspark.sql.functions import col
    
    df = df\
    .withColumn("passenger_count", col("passenger_count").cast(IntegerType())) \
    .withColumn("total_amount", col("total_amount").cast(FloatType())) \
    .withColumn("tip_amount", col("tip_amount").cast(FloatType())) \
    .withColumn("trip_distance", col("trip_distance").cast(FloatType())) \
    .withColumn("fare_amount", col("fare_amount").cast(FloatType())) \
    .withColumn("tpep_pickup_datetime", col("tpep_pickup_datetime").cast(TimestampType()))\
    .withColumn("tpep_dropoff_datetime", col("tpep_dropoff_datetime").cast(TimestampType()))
   
    # END YOUR CODE HERE -----------
    
    return df
#dfCast = clean_data(df)
#print("After Casting columns")
#dfCast.printSchema()



### Q1.2

Find rate per person for based on how many passengers travel between pickup and dropoff locations. 

In [5]:
#export

from pyspark.sql.functions import sum, col

def common_pair(df):
    '''
    input: df a dataframe
    output: df a dataframe with following columns:
            - PULocationID
            - DOLocationID
            - total_passenger_count
            - per_person_rate
            
    per_person_rate is the total_amount per person for a given pair.
    
    '''

    # START YOUR CODE HERE ---------

    # Eliminate cases where the pickup and dropoff are the same
    df_filtered = df.filter(col("PULocationID") != col("DOLocationID"))

    # We will use PySpark aggregate function
    # the aggregation is basically to get passenger_count, per_person_rate for the pair pickup-dropoff location
    # per_person_rate = total_amount/total_passenger_count
    # a. groupBy Pickup and Dropoff locations
    # b. for each unique pickup and dropoff we will aggregate the data
    # c. add all passenger counts for a given pickup/dropoff pair.
    # d. calculate the total per person rate
    
    df_pairs = df_filtered.groupBy("PULocationID", "DOLocationID") \
        .agg(sum(col("passenger_count")).alias("total_passenger_count"),
            (sum(col("total_amount")) / sum(col("passenger_count"))).alias("per_person_rate"))

    df_result = df_pairs.orderBy(col("total_passenger_count").desc(), col("per_person_rate").desc()) \
                        .limit(10)

    df = df_result
    # END YOUR CODE HERE -----------
    
    return df

### Q1.3

Find trips which trip distances generate the highest tip percentage.

In [6]:
#export

# df = load_data()
# print("Before Casting columns")
# df.printSchema()
# dfCast = clean_data(df)
# print("After Casting columns")
# dfCast.printSchema()
# dfCommon = common_pair(dfCast)
# print("After Aggregation")
# dfCommon.printSchema()
# dfCommon.show(10)
# distance_with_most_tip(df)

from pyspark.sql.functions import col, round, avg, ceil


def distance_with_most_tip(df):
    '''
    input: df a dataframe
    output: df a dataframe with following columns:
            - trip_distance
            - tip_percent
            
    trip_percent is the percent of tip out of fare_amount
    
    '''
    
    # START YOUR CODE HERE ---------
    # Only include trip_distance greater than 0
    df_filtered = df.filter((col("fare_amount") > 2.00) & (col("trip_distance") > 0))

    # calculate tip percentatge
    # round it to the nearest mile
    df_tips = df_filtered.withColumn("tip_percent", ceil((col("tip_amount") * 100) / col("fare_amount")))
                                     
    #                         .withColumn("trip_distance", round(col("trip_distance")))

    # round tips
    df_tips = df_tips.withColumn("trip_distance", round(col("trip_distance")))
    
    # groupBy trip_distance
    # use aggregation to calculate average tip_percent
    df_grouped = df_tips.groupBy("trip_distance") \
                        .agg(avg("tip_percent").alias("tip_percent"))

    # sort Descending tip_percent
    # Select top 15

    df = df_grouped.orderBy(col("tip_percent").desc()).limit(15)
    
    # END YOUR CODE HERE -----------
        
    
    return df

In [7]:
# df = load_data()
# # print("Before Casting columns")
# # df.printSchema()
# dfCast = clean_data(df)
# # print("After Casting columns")
# # dfCast.printSchema()
# dfCommon = common_pair(dfCast)
# # print("After Aggregation")
# # dfCommon.printSchema()
# dfCommon.show(10)
# # distance_with_most_tip(dfCommon)
# df_tip = distance_with_most_tip(df)
# df_tip.show()

### Q1.4

Determine the average speed at different times of day.

In [8]:
#export

from pyspark.sql.functions import col, hour, unix_timestamp, avg, when, date_format

def time_with_most_traffic(df):
    '''
    input: df a dataframe
    output: df a dataframe with following columns:
            - time_of_day
            - am_avg_speed
            - pm_avg_speed
            
    am_avg_speed and pm_avg_speed are the average trip distance / average trip time calculated for each hour
    
    '''
    
    # START YOUR CODE HERE ---------
    # Only filter trips that have trip_distance gt 0
    # df_filtered = df.filter(col("trip_distance") > 0)
    df_filtered = df.filter((col("trip_distance") > 0) & 
                            col("tpep_pickup_datetime").isNotNull() & 
                            col("tpep_dropoff_datetime").isNotNull())
    
    # extract  hour from pickup timestamp
    df_time = df_filtered.withColumn("hour", hour(col("tpep_pickup_datetime")))

    # calculate trip_duration in hours
    df_time = df_time.withColumn("trip_duration", 
        (unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 3600)

    # Formats "1 AM", "12 PM", ...
    df_time = df_time \
            .withColumn("time_of_day", date_format(col("tpep_pickup_datetime"), "h a")) \
            .withColumn("am_pm", when(col("hour") < 12, "AM").otherwise("PM"))



    # df_time = df_time.withColumn("time_of_day", 
    #     when(col("hour") == 0, 12)
    #    .when(col("hour") > 12, col("hour") - 12)
    #    .otherwise(col("hour")))


    # Separate AM/PM
    # df_time = df_time.withColumn("am_pm", when(col("hour") < 12, "AM").otherwise("PM"))
    
    # convert 24 hours to 12 hr format. 
    df_time = df_time.withColumn("time_of_day", when(col("hour") == 0, 12)  # convert midnight to 12 AM
                                              .when(col("hour") > 12, col("hour") - 12)  # convert 13 - 23 to 1-11 PM
                                              .otherwise(col("hour")))  # Keep 1-11 as-is
   
    # calculate average duration of each trip and average trip distance groupBy hour
    df_avg = df_time.groupBy("time_of_day", "am_pm") \
                    .agg(
                        avg("trip_duration").alias("avg_duration"),
                        avg("trip_distance").alias("avg_distance")
                    )    
    
    
    # calculate  average speed / hr
    # df_avg = df_avg.withColumn("avg_speed", col("avg_distance") / col("avg_duration"))
    df_avg = df_avg.withColumn("avg_speed", 
                               when(col("avg_duration") > 0, col("avg_distance") / col("avg_duration"))
                               .otherwise(None))
    
    
    # Pivot to create AM & PM speed columns.
    df_result = df_avg.groupBy("time_of_day") \
                        .pivot("am_pm", ["AM", "PM"]) \
                        .agg(avg("avg_speed").alias("avg_speed")) \
                        .orderBy("time_of_day")

    if "AM" in df_result.columns:
        df_result = df_result.withColumnRenamed("AM", "am_avg_speed")
    if "PM" in df_result.columns:
        df_result = df_result.withColumnRenamed("PM", "pm_avg_speed")

    
   
    # df = df_result.limit(10)
    df = df_result.orderBy(col("time_of_day").asc()).limit(10)

    # 
    # END YOUR CODE HERE -----------
    
    return df

## The below cells are for you to investigate your solutions and will not be graded

In [9]:
df = load_data()
df = clean_data(df)

In [10]:
common_pair(df).show()

+------------+------------+---------------------+------------------+
|PULocationID|DOLocationID|total_passenger_count|   per_person_rate|
+------------+------------+---------------------+------------------+
|         239|         238|                   62|  4.26274198870505|
|         237|         236|                   60| 4.482500068346659|
|         263|         141|                   52|3.4190384974846473|
|         161|         236|                   42| 5.368571440378825|
|         148|          79|                   42| 4.711904752822149|
|         142|         238|                   39|  5.05487182812813|
|         141|         236|                   37| 4.355675723101641|
|         239|         143|                   37| 4.252162224537617|
|         239|         142|                   35| 3.817714350564139|
|          79|         170|                   34| 6.394705884596881|
+------------+------------+---------------------+------------------+



In [11]:
distance_with_most_tip(df).show()

+-------------+------------------+
|trip_distance|       tip_percent|
+-------------+------------------+
|          1.0|17.202070207020704|
|          0.0|  15.6578073089701|
|          2.0| 15.62034574468085|
|         17.0|15.538461538461538|
|          5.0|15.139318885448917|
|          3.0|14.779376498800959|
|         21.0|14.666666666666666|
|         19.0|14.357142857142858|
|          4.0|13.844181459566075|
|          9.0|13.838235294117647|
|          6.0|13.654867256637168|
|          8.0|12.172413793103448|
|         23.0|              12.0|
|         18.0|             11.75|
|         10.0|              11.7|
+-------------+------------------+



In [12]:
time_with_most_traffic(df).show()

+-----------+------------------+-------------------+
|time_of_day|      am_avg_speed|       pm_avg_speed|
+-----------+------------------+-------------------+
|          1|10.845483413697353|  8.972972792548102|
|          5|              NULL| 0.5137660239764732|
|          6|              NULL|  9.989847870647605|
|          7|              NULL|0.18415305490417713|
|          8|              NULL| 0.5183127622697896|
|         10|              NULL| 0.6147483972627696|
|         11|              NULL|  4.651171294815548|
|         12| 9.382686366547683|               NULL|
+-----------+------------------+-------------------+

